# Interpolation method - break down steps to debug
* When finding nearest vp, there are stops that have later arrival times coming before earlier arrivals.
   * stop 1 arrives after stop 2. this can't happen in real life.
   * filter down to these stops and try again
   * how do we distinguish between whether stop 1 or stop 2 is the "correct" one?
* Want to add direction to deal with loop or inlining shapes --> maybe this can be expanded to all shapes
   * Do not even allow opposite direction vp to be selected as nearest vp

In [1]:
import dask.dataframe as dd
import numpy as np
import pandas as pd

from segment_speed_utils import helpers, wrangle_shapes
from segment_speed_utils.project_vars import SEGMENT_GCS, PROJECT_CRS 
                                              
from shared_utils import rt_dates

analysis_date = rt_dates.DATES["sep2023"]

STOP_SEG_DICT = helpers.get_parameters(
    "./scripts/config.yml", "stop_segments")

## Between stops, arrival times behaving not as expected
There are erroneous calculations here.

Prior arrival time can't take place **after** arrival time. 

Ex: `trip_instance_key == "0000412e74a0e91993ce66bcbc4e3e73"`, stop 1's nearest position is after stop 2's nearest position.

Handle these along with loop or inlining shapes.

In [2]:
NEAREST_VP = f"{STOP_SEG_DICT['stage2']}_error_{analysis_date}"
STOP_ARRIVALS = f"{STOP_SEG_DICT['stage3']}_{analysis_date}"

df = pd.read_parquet(
    f"{SEGMENT_GCS}projection/{NEAREST_VP}.parquet"
)

In [3]:
df.columns

Index(['trip_instance_key', 'shape_array_key', 'stop_sequence', 'stop_id',
       'stop_meters', 'nearest_vp_idx', 'subseq_vp_idx',
       'rolling_nearest_vp_idx', 'nearest_vp_idx_monotonic',
       'rolling_stop_meters', 'stop_meters_monotonic'],
      dtype='object')

In [ ]:
df.nearest_vp_idx_monotonic.value_counts()

In [ ]:
df.stop_meters_monotonic.value_counts()

In [ ]:
df[(df.nearest_vp_idx_monotonic==False) & 
   (df.stop_meters_monotonic==False)].shape

In [ ]:
pd.crosstab(df.nearest_vp_idx_monotonic, 
            df.stop_meters_monotonic, normalize=True)

In [ ]:
df[(df.nearest_vp_idx_monotonic==False) &
    (df.stop_meters_monotonic==True)]

In [ ]:
df[df.stop_meters_monotonic==False][[
    "stop_sequence", 
    "nearest_vp_idx",
    "rolling_nearest_vp_idx", "nearest_vp_idx_monotonic", 
    "stop_meters", "rolling_stop_meters", 
    "stop_meters_monotonic"
]]

In [ ]:
grouped_df = df.groupby("trip_instance_key", 
                        observed=True, group_keys=False)
min_max_stop = pd.merge(
    grouped_df.agg({"stop_sequence": "min"}).rename(
        columns = {"stop_sequence": "min_stop"}),
    grouped_df.agg({"stop_sequence": "max"}).rename(
        columns = {"stop_sequence": "max_stop"}),
    on = "trip_instance_key",
    how = "inner"
).reset_index()

In [ ]:
df = pd.merge(
    df,
    min_max_stop,
    on = "trip_instance_key",
    how = "inner"
)

In [4]:
stop_arrivals = pd.read_parquet(
    f"{SEGMENT_GCS}{STOP_ARRIVALS}.parquet",
    columns = ["trip_instance_key", "stop_sequence", "arrival_time"]
)

In [5]:
trips_monotonicity = (stop_arrivals.groupby("trip_instance_key")
                       .arrival_time
                       .is_monotonic_increasing
                      ).to_frame().reset_index()

trips_monotonicity

,trip_instance_key,arrival_time
0,0000412e74a0e91993ce66bcbc4e3e73,False
1,00019686e6c7bf335148c8d290feb285,False
2,0001ad7e1ef246cf6d68599de0fdcaad,False
3,0002738d89e3c4a4ac7ad0bc330ca764,False
4,0003c9b5cf6f1383e81d20410c8c370c,True
...,...,...
67892,fffdede1d9b915471171b195b38936bf,False
67893,fffdffc7b5993e1ac5e713e24f58769b,False
67894,fffe208e64d2716649bc71058248d9f5,True
67895,fffe87969abd39288b39934ebd55390f,False


In [6]:
trips_monotonicity.arrival_time.value_counts()

False    40008
True     27889
Name: arrival_time, dtype: int64

In [7]:
fail_trips = trips_monotonicity[
    trips_monotonicity.arrival_time==False
].sample(25).trip_instance_key.unique()

In [8]:
stop_arrivals[stop_arrivals.trip_instance_key==fail_trips[7]]

,trip_instance_key,stop_sequence,arrival_time
671076,3e34526ed9aedbdfb6ad8d357877cf60,1,2023-09-13 18:02:08
671077,3e34526ed9aedbdfb6ad8d357877cf60,2,2023-09-13 17:51:46
671078,3e34526ed9aedbdfb6ad8d357877cf60,3,2023-09-13 17:53:26
671079,3e34526ed9aedbdfb6ad8d357877cf60,4,2023-09-13 17:54:07
671080,3e34526ed9aedbdfb6ad8d357877cf60,5,2023-09-13 17:56:51
671081,3e34526ed9aedbdfb6ad8d357877cf60,6,2023-09-13 17:57:22
671082,3e34526ed9aedbdfb6ad8d357877cf60,7,2023-09-13 17:57:50
671083,3e34526ed9aedbdfb6ad8d357877cf60,8,2023-09-13 17:58:14
671084,3e34526ed9aedbdfb6ad8d357877cf60,9,2023-09-13 18:02:08


In [9]:
import altair as alt

def plot_stop_arrivals(df, one_trip):
    chart = (alt.Chart(df[df.trip_instance_key==one_trip])
             .mark_line()
             .encode(
                 x="stop_sequence",
                 y="arrival_time:T",
                 tooltip=["stop_sequence", "arrival_time"]
             ).properties(title=one_trip)
             .interactive()
            )
    
    display(chart)

In [10]:
for t in fail_trips:
    plot_stop_arrivals(stop_arrivals, t)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

...

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

In [11]:
#trip_stats.sample(10).trip_instance_key.unique()
subset_trip_keys = [
    '9fad69264acd8387150f45b27d4b2d09',
    '44a55d2fa2588a479065ef7702475ef1',
    '36070a2428e62b96368d072eb2a8fc1b',
    '7f665900c6b0879f4b9bda43b93fefe3',
    '8e8ba9993d52388539d06a46710c1dbc',
    'b301c2170c1ca49bbc1a9b600cccf643',
    '9373f5b0de977a718dea50fd90443619',
    '8415b3949147c9dc3d5ceb37863440b1',
    '984f598419c1d0830ef4618d495c1bd7',
    '815e4dd921cdcb61ad2dbb1ca5f08a39'
]

In [12]:
df2 = pd.merge(
    df,
    stop_arrivals,
    on = ["trip_instance_key", "stop_sequence"],
    how = "inner"
)

In [15]:
df2[df2.trip_instance_key=="00019686e6c7bf335148c8d290feb285"]

,trip_instance_key,shape_array_key,stop_sequence,stop_id,stop_meters,nearest_vp_idx,subseq_vp_idx,rolling_nearest_vp_idx,nearest_vp_idx_monotonic,rolling_stop_meters,stop_meters_monotonic,arrival_time
26,00019686e6c7bf335148c8d290feb285,c7d45bdd03e554c41e896bc0e8401093,1,e37ef534-cce7-4470-84b4-25ba98c51114,0.163113,2229819.0,2229819.0,"[2229819.0, 2229819.0]",True,"[0.16311274941444126, 1161.286179681159]",True,2023-09-13 13:59:56
27,00019686e6c7bf335148c8d290feb285,c7d45bdd03e554c41e896bc0e8401093,2,688c492c-6387-4c3b-98d3-af3566f59afe,1161.286180,2229819.0,2229819.0,"[2229819.0, 2229819.0, 2229818.0]",False,"[0.16311274941444126, 1161.286179681159, 1546....",True,2023-09-13 13:59:56
28,00019686e6c7bf335148c8d290feb285,c7d45bdd03e554c41e896bc0e8401093,3,6a41f1d7-ec6b-4adc-8ff0-574013f93c75,1546.668769,2229818.0,2229819.0,"[2229819.0, 2229818.0, 2229724.0]",False,"[1161.286179681159, 1546.6687686280013, 1970.8...",True,2023-09-13 13:59:56
29,00019686e6c7bf335148c8d290feb285,c7d45bdd03e554c41e896bc0e8401093,4,d69a605a-28f5-4e67-a821-e1cb95582193,1970.847443,2229724.0,2229725.0,"[2229818.0, 2229724.0, 2229726.0]",False,"[1546.6687686280013, 1970.847443482172, 2352.1...",True,2023-09-13 13:28:29
30,00019686e6c7bf335148c8d290feb285,c7d45bdd03e554c41e896bc0e8401093,5,620a5e57-6afc-426a-beff-2a153b6df22a,2352.160315,2229726.0,2229727.0,"[2229724.0, 2229726.0, 2229729.0]",True,"[1970.847443482172, 2352.160315248948, 2762.35...",True,2023-09-13 13:29:07
31,00019686e6c7bf335148c8d290feb285,c7d45bdd03e554c41e896bc0e8401093,6,382d4f18-e95f-46bf-b9a9-aaaa735c37c8,2762.359792,2229729.0,2229730.0,"[2229726.0, 2229729.0, 2229733.0]",True,"[2352.160315248948, 2762.3597919880635, 3314.7...",True,2023-09-13 13:30:16
32,00019686e6c7bf335148c8d290feb285,c7d45bdd03e554c41e896bc0e8401093,7,80b1b724-76bd-4159-9619-e052e647fcc8,3314.712655,2229733.0,2229734.0,"[2229729.0, 2229733.0, 2229737.0]",True,"[2762.3597919880635, 3314.7126547877156, 3826....",True,2023-09-13 13:31:28
33,00019686e6c7bf335148c8d290feb285,c7d45bdd03e554c41e896bc0e8401093,8,09b664e3-2ead-4c9e-8a53-3e4d2127cb01,3826.926504,2229737.0,2229738.0,"[2229733.0, 2229737.0, 2229767.0]",True,"[3314.7126547877156, 3826.92650362235, 4204.13...",True,2023-09-13 13:32:56
34,00019686e6c7bf335148c8d290feb285,c7d45bdd03e554c41e896bc0e8401093,9,2133417c-1449-47cd-b2a9-960dfe208ecb,4204.138228,2229767.0,2229768.0,"[2229737.0, 2229767.0, 2229769.0]",True,"[3826.92650362235, 4204.138228326698, 4587.505...",True,2023-09-13 13:42:53
35,00019686e6c7bf335148c8d290feb285,c7d45bdd03e554c41e896bc0e8401093,10,effe7cd2-819b-4c55-bb86-531e0bd55dac,4587.505534,2229769.0,2229770.0,"[2229767.0, 2229769.0, 2229771.0]",True,"[4204.138228326698, 4587.505533926202, 5111.69...",True,2023-09-13 13:43:23


In [16]:
df2[df2.trip_instance_key=="0001ad7e1ef246cf6d68599de0fdcaad"
   ].tail(10)

,trip_instance_key,shape_array_key,stop_sequence,stop_id,stop_meters,nearest_vp_idx,subseq_vp_idx,rolling_nearest_vp_idx,nearest_vp_idx_monotonic,rolling_stop_meters,stop_meters_monotonic,arrival_time
59,0001ad7e1ef246cf6d68599de0fdcaad,4b7b4aa51e58202f4aa066386bd93699,21,dea3db20-1108-4028-b7de-931c667b189e,10078.393942,2202485.0,2202486.0,"[2202485.0, 2202485.0, 2202449.0]",False,"[6305.515211243769, 10078.393941738626, 5481.6...",False,2023-09-13 17:18:49
60,0001ad7e1ef246cf6d68599de0fdcaad,4b7b4aa51e58202f4aa066386bd93699,22,349f5c2b-f9cc-42aa-ac08-82eedf55e6ad,5481.655022,2202449.0,2202450.0,"[2202485.0, 2202449.0, 2202485.0]",False,"[10078.393941738626, 5481.655022188139, 10853....",False,2023-09-13 17:06:59
61,0001ad7e1ef246cf6d68599de0fdcaad,4b7b4aa51e58202f4aa066386bd93699,23,13288a16-200c-4f43-aaf2-2ea2234b25fa,10853.162912,2202485.0,2202486.0,"[2202449.0, 2202485.0, 2202492.0]",True,"[5481.655022188139, 10853.162911846139, 11372....",True,2023-09-13 17:18:51
62,0001ad7e1ef246cf6d68599de0fdcaad,4b7b4aa51e58202f4aa066386bd93699,24,ce1eac51-5bb0-4185-9dac-54f4349b186c,11372.198148,2202492.0,2202493.0,"[2202485.0, 2202492.0, 2202493.0]",True,"[10853.162911846139, 11372.198148020185, 11579...",True,2023-09-13 17:21:05
63,0001ad7e1ef246cf6d68599de0fdcaad,4b7b4aa51e58202f4aa066386bd93699,25,8082a24a-406e-4f8c-8fac-a5bf7eefb08c,11579.663731,2202493.0,2202494.0,"[2202492.0, 2202493.0, 2202495.0]",True,"[11372.198148020185, 11579.663731311422, 11921...",True,2023-09-13 17:21:28
64,0001ad7e1ef246cf6d68599de0fdcaad,4b7b4aa51e58202f4aa066386bd93699,26,7202e872-1a4a-45d3-b671-20688f775d3b,11921.052645,2202495.0,2202496.0,"[2202493.0, 2202495.0, 2202496.0]",True,"[11579.663731311422, 11921.052645013666, 12169...",True,2023-09-13 17:22:09
65,0001ad7e1ef246cf6d68599de0fdcaad,4b7b4aa51e58202f4aa066386bd93699,27,543f4694-437d-4a8d-a857-c32ba539b85f,12169.348749,2202496.0,2202497.0,"[2202495.0, 2202496.0, 2202500.0]",True,"[11921.052645013666, 12169.348748696199, 12755...",True,2023-09-13 17:22:30
66,0001ad7e1ef246cf6d68599de0fdcaad,4b7b4aa51e58202f4aa066386bd93699,28,40311c02-6187-42e4-b8d6-63ccf7d373f1,12755.771383,2202500.0,2202501.0,"[2202496.0, 2202500.0, 2202504.0]",True,"[12169.348748696199, 12755.771383059999, 12908...",True,2023-09-13 17:23:54
67,0001ad7e1ef246cf6d68599de0fdcaad,4b7b4aa51e58202f4aa066386bd93699,29,08e5dff7-bd3f-4595-9b2b-37205bfa29bb,12908.495855,2202504.0,2202505.0,"[2202500.0, 2202504.0, 2202536.0]",True,"[12755.771383059999, 12908.495854575252, 0.169...",False,2023-09-13 17:25:02
68,0001ad7e1ef246cf6d68599de0fdcaad,4b7b4aa51e58202f4aa066386bd93699,30,52c2636c-34a3-434c-99ae-cdf3dc36d15c,0.169502,2202536.0,2202536.0,"[2202504.0, 2202536.0]",True,"[12908.495854575252, 0.16950179216999356]",False,2023-09-13 17:35:33


Nearly half of the arrival errors are surrounded by stops where arrival time is correct.

Let's hit this first and then deal with endpoints.

If almost every trip has errors, it matters where the arrival order error comes in. Must take the minimum stop sequence of a clear monotonic period and set that as the max to filter out vp_idx that occur afterwards.

In [18]:
import altair as alt

for t in subset_trip_keys:
    
    subset_df = df[df.trip_instance_key==t]
    
    chart = (alt.Chart(subset_df)
             .mark_line()
             .encode(
                 x="stop_sequence",
                 y="nearest_vp_idx_monotonic"
             ).properties(title=f"{t}")
    )
    display(chart)
    
    chart2 = (alt.Chart(subset_df[subset_df.nearest_vp_idx_monotonic == True])
              .mark_line()
              .encode(
                  x="stop_sequence",
                  y="stop_meters_monotonic"
              )
    )
    display(chart2)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

...

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

In [ ]:
nearest_array = df.groupby("trip_instance_key").agg(
    {"nearest_vp_idx": lambda x: list(x)}).reset_index()

In [ ]:
nearest_array = nearest_array.assign(
    array_diff = nearest_array.apply(
        lambda x: 
        np.ediff1d(np.asarray(x.nearest_vp_idx)),
        axis=1)
)

In [ ]:
nearest_array = nearest_array.assign(
    wrong_times = nearest_array.apply(
        lambda x: 1 if len(np.where(x.array_diff < 0)[0] > 0)
        else 0, axis=1
    )
)

In [ ]:
nearest_array.wrong_times.value_counts()

## Index into specific portions of array

In [ ]:
one_trip = "bf87a17838cdaff5ba78fb70edd4f1bb"

In [ ]:
def rt_trips_to_shape(analysis_date: str) -> pd.DataFrame:
    """
    Filter down trip_instance_keys from schedule to 
    trips present in vp.
    Provide shape_array_key associated with trip_instance_key.
    """
    # Get RT trips
    rt_trips = pd.read_parquet(
        f"{SEGMENT_GCS}vp_usable_{analysis_date}",
        filters = [[("trip_instance_key", "==", one_trip)]],
        columns = ["trip_instance_key"]
    ).drop_duplicates()

    # Find the shape_array_key for RT trips
    trip_to_shape = helpers.import_scheduled_trips(
        analysis_date,
        columns = ["trip_instance_key", "shape_array_key"],
        get_pandas = True
    ).merge(
        rt_trips,
        on = "trip_instance_key",
        how = "inner"
    )

    # Find whether it's loop or inlining
    shapes_loop_inlining = pd.read_parquet(
        f"{SEGMENT_GCS}stops_projected_{analysis_date}.parquet",
        columns = [
            "shape_array_key", "loop_or_inlining", 
            "stop_primary_direction", 
        ]
    ).drop_duplicates().merge(
        trip_to_shape,
        on = "shape_array_key",
        how = "inner"
    )
    
    return shapes_loop_inlining

In [ ]:
trip_shape_crosswalk = rt_trips_to_shape(analysis_date)

In [ ]:
vp = pd.read_parquet(
    f"{SEGMENT_GCS}vp_usable_{analysis_date}",
    filters = [[("trip_instance_key", "==", one_trip)]],
    columns = ["trip_instance_key", "vp_idx", 
                "vp_primary_direction", 
                  ]
    )

In [ ]:
subset_vp = vp.vp_idx.unique()

In [ ]:
projected_shape_meters = pd.read_parquet(
    f"{SEGMENT_GCS}projection/vp_projected_{analysis_date}.parquet",
    filters = [[("vp_idx", "in", subset_vp)]]
)

vp_with_projection = pd.merge(
    vp,
    projected_shape_meters,
    on = "vp_idx",
    how = "inner"
).merge(
    trip_shape_crosswalk,
        on = "trip_instance_key",
        how = "inner"
    )

In [ ]:
shape_keys = trip_shape_crosswalk.shape_array_key.unique()

stops_projected = pd.read_parquet(
    f"{SEGMENT_GCS}stops_projected_{analysis_date}.parquet",
    filters = [[("shape_array_key", "in", shape_keys)]],
    columns = ["shape_array_key", "stop_sequence", "stop_id", 
               "shape_meters", "stop_primary_direction"]
).rename(columns = {"shape_meters": "stop_meters"})

In [ ]:
trip_shape_cols = ["trip_instance_key", "shape_array_key"]

trip_info = (
    vp_with_projection
    .groupby(trip_shape_cols, 
              observed=True, group_keys=False)
    .agg({
        "vp_idx": lambda x: list(x),
        "shape_meters": lambda x: list(x),
        "vp_primary_direction": lambda x: list(x),
    })
    .reset_index()
    .rename(columns = {
        "vp_idx": "vp_idx_arr",
        "shape_meters": "shape_meters_arr",
        "vp_primary_direction": "vp_dir_arr"
    })
)

In [ ]:
vp_to_stop = pd.merge(
    trip_info,
    stops_projected,
    on = "shape_array_key",
    how = "inner"
)

In [ ]:
this_stop_direction = vp_to_stop.stop_primary_direction.iloc[0]
this_stop_meters = vp_to_stop.stop_meters.iloc[0]

In [ ]:
this_stop_direction

In [ ]:
opposite_to_stop_direction = wrangle_shapes.OPPOSITE_DIRECTIONS[
    this_stop_direction]

In [ ]:
opposite_to_stop_direction

In [ ]:
vp_dir_array = np.asarray(vp_to_stop.vp_dir_arr.iloc[0])
shape_meters_array = np.asarray(vp_to_stop.shape_meters_arr.iloc[0])
vp_meters_array = np.asarray(vp_to_stop.vp_idx_arr.iloc[0])

In [ ]:
#https://stackoverflow.com/questions/16094563/numpy-get-index-where-value-is-true
valid_vp_idx_indices = (vp_dir_array != opposite_to_stop_direction).nonzero()

In [ ]:
vp_meters_array[valid_vp_idx_indices]

In [ ]:
for row in vp_to_stop.tail(1).itertuples():
    this_stop_meters = np.asarray(getattr(row, "vp_idx_arr"))[valid_vp_idx_indices]
    #valid_stop_meters = this_stop_meters
    #print(valid_stop_meters)
    print(this_stop_meters)

## Plot interpolated arrivals -> speed

In [ ]:
df = pd.read_parquet(
    f"{SEGMENT_GCS}stop_arrivals_speed_{analysis_date}_2.parquet")

In [ ]:
bins = range(0, 75, 5)

df[df.speed_mph < 80].hist(
    "speed_mph", 
     bins = bins)